## Import

In [1]:
import os
import cv2
import random
import shutil
import numpy as np
import PIL
from matplotlib import pyplot as plt
from copy import deepcopy
import serial
import time
import pandas as pd
import json

## Functions

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

def show(img):
    out = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(1,1))
    plt.imshow(out)
    plt.show()

In [3]:
def rgb2hsv(rgb):
    """ convert RGB to HSV color space

    :param rgb: np.ndarray
    :return: np.ndarray
    """

    rgb = rgb.astype('float')
    maxv = np.amax(rgb, axis=2)
    maxc = np.argmax(rgb, axis=2)
    minv = np.amin(rgb, axis=2)
    minc = np.argmin(rgb, axis=2)

    hsv = np.zeros(rgb.shape, dtype='float')
    hsv[maxc == minc, 0] = np.zeros(hsv[maxc == minc, 0].shape)
    hsv[maxc == 0, 0] = (((rgb[..., 1] - rgb[..., 2]) * 60.0 / (maxv - minv + np.spacing(1))) % 360.0)[maxc == 0]
    hsv[maxc == 1, 0] = (((rgb[..., 2] - rgb[..., 0]) * 60.0 / (maxv - minv + np.spacing(1))) + 120.0)[maxc == 1]
    hsv[maxc == 2, 0] = (((rgb[..., 0] - rgb[..., 1]) * 60.0 / (maxv - minv + np.spacing(1))) + 240.0)[maxc == 2]
    hsv[maxv == 0, 1] = np.zeros(hsv[maxv == 0, 1].shape)
    hsv[maxv != 0, 1] = (1 - minv / (maxv + np.spacing(1)))[maxv != 0]
    hsv[..., 2] = maxv

    return hsv

def hsv2rgb(hsv):
    """ convert HSV to RGB color space

    :param hsv: np.ndarray
    :return: np.ndarray
    """

    hi = np.floor(hsv[..., 0] / 60.0) % 6
    hi = hi.astype('uint8')
    v = hsv[..., 2].astype('float')
    f = (hsv[..., 0] / 60.0) - np.floor(hsv[..., 0] / 60.0)
    p = v * (1.0 - hsv[..., 1])
    q = v * (1.0 - (f * hsv[..., 1]))
    t = v * (1.0 - ((1.0 - f) * hsv[..., 1]))

    rgb = np.zeros(hsv.shape)
    rgb[hi == 0, :] = np.dstack((v, t, p))[hi == 0, :]
    rgb[hi == 1, :] = np.dstack((q, v, p))[hi == 1, :]
    rgb[hi == 2, :] = np.dstack((p, v, t))[hi == 2, :]
    rgb[hi == 3, :] = np.dstack((p, q, v))[hi == 3, :]
    rgb[hi == 4, :] = np.dstack((t, p, v))[hi == 4, :]
    rgb[hi == 5, :] = np.dstack((v, p, q))[hi == 5, :]

    return rgb

## rubik cube simulator and solver

In [20]:
class rubik:
    _map_st = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
    cube = []
    def __init__(self):
        self.moveslist = ""
        self._l = [0, 31, 2, 3, 34, 5, 6, 37, 8, 9, 10, 1, 12, 13, 4, 15, 16, 7, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 51, 32, 33, 54, 35, 36, 57, 38, 39, 40, 47, 44, 41, 48, 45, 42, 49, 46, 43, 50, 11, 52, 53, 14, 55, 56, 17, 58, 59]
        self._r = [0, 1, 2, 13, 4, 5, 16, 7, 8, 19, 10, 11, 12, 53, 14, 15, 56, 17, 18, 59, 20, 27, 24, 21, 28, 25, 22, 29, 26, 23, 30, 31, 32, 3, 34, 35, 6, 37, 38, 9, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 33, 54, 55, 36, 57, 58, 39]
        self._u = [0, 21, 22, 23, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 59, 58, 57, 24, 25, 26, 27, 28, 29, 30, 37, 34, 31, 38, 35, 32, 39, 36, 33, 40, 1, 2, 3, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 43, 42, 41]
        self._d = [0, 1, 2, 3, 4, 5, 6, 47, 48, 49, 10, 17, 14, 11, 18, 15, 12, 19, 16, 13, 20, 21, 22, 23, 24, 25, 26, 7, 8, 9, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 53, 52, 51, 50, 29, 28, 27, 54, 55, 56, 57, 58, 59]
        self._f = [0, 7, 4, 1, 8, 5, 2, 9, 6, 3, 10, 27, 24, 21, 14, 15, 16, 17, 18, 19, 20, 37, 22, 23, 38, 25, 26, 39, 28, 29, 30, 31, 32, 33, 34, 35, 36, 49, 46, 43, 40, 41, 42, 11, 44, 45, 12, 47, 48, 13, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
        self._b = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 41, 44, 47, 20, 21, 22, 19, 24, 25, 18, 27, 28, 17, 30, 23, 26, 29, 34, 35, 36, 37, 38, 39, 40, 33, 42, 43, 32, 45, 46, 31, 48, 49, 50, 57, 54, 51, 58, 55, 52, 59, 56, 53]
        self._x = [0, 11, 12, 13, 14, 15, 16, 17, 18, 19, 10, 51, 52, 53, 54, 55, 56, 57, 58, 59, 20, 27, 24, 21, 28, 25, 22, 29, 26, 23, 30, 1, 2, 3, 4, 5, 6, 7, 8, 9, 40, 43, 46, 49, 42, 45, 48, 41, 44, 47, 50, 31, 32, 33, 34, 35, 36, 37, 38, 39]
        self._y = [0, 21, 22, 23, 24, 25, 26, 27, 28, 29, 10, 13, 16, 19, 12, 15, 18, 11, 14, 17, 20, 59, 58, 57, 56, 55, 54, 53, 52, 51, 30, 37, 34, 31, 38, 35, 32, 39, 36, 33, 40, 1, 2, 3, 4, 5, 6, 7, 8, 9, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41]
        self._z = [0, 7, 4, 1, 8, 5, 2, 9, 6, 3, 10, 27, 24, 21, 28, 25, 22, 29, 26, 23, 20, 37, 34, 31, 38, 35, 32, 39, 36, 33, 30, 47, 44, 41, 48, 45, 42, 49, 46, 43, 40, 17, 14, 11, 18, 15, 12, 19, 16, 13, 50, 53, 56, 59, 52, 55, 58, 51, 54, 57]
        self._L = [0, 11, 2, 3, 14, 5, 6, 17, 8, 9, 10, 51, 12, 13, 54, 15, 16, 57, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 1, 32, 33, 4, 35, 36, 7, 38, 39, 40, 43, 46, 49, 42, 45, 48, 41, 44, 47, 50, 31, 52, 53, 34, 55, 56, 37, 58, 59]
        self._R = [0, 1, 2, 33, 4, 5, 36, 7, 8, 39, 10, 11, 12, 3, 14, 15, 6, 17, 18, 9, 20, 23, 26, 29, 22, 25, 28, 21, 24, 27, 30, 31, 32, 53, 34, 35, 56, 37, 38, 59, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 13, 54, 55, 16, 57, 58, 19]
        self._U = [0, 41, 42, 43, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 1, 2, 3, 24, 25, 26, 27, 28, 29, 30, 33, 36, 39, 32, 35, 38, 31, 34, 37, 40, 59, 58, 57, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 23, 22, 21]
        self._D = [0, 1, 2, 3, 4, 5, 6, 27, 28, 29, 10, 13, 16, 19, 12, 15, 18, 11, 14, 17, 20, 21, 22, 23, 24, 25, 26, 53, 52, 51, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 7, 8, 9, 50, 49, 48, 47, 54, 55, 56, 57, 58, 59]
        self._F = [0, 3, 6, 9, 2, 5, 8, 1, 4, 7, 10, 43, 46, 49, 14, 15, 16, 17, 18, 19, 20, 13, 22, 23, 12, 25, 26, 11, 28, 29, 30, 31, 32, 33, 34, 35, 36, 21, 24, 27, 40, 41, 42, 39, 44, 45, 38, 47, 48, 37, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
        self._B = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 29, 26, 23, 20, 21, 22, 31, 24, 25, 32, 27, 28, 33, 30, 47, 44, 41, 34, 35, 36, 37, 38, 39, 40, 17, 42, 43, 18, 45, 46, 19, 48, 49, 50, 53, 56, 59, 52, 55, 58, 51, 54, 57]
        self._X = [0, 31, 32, 33, 34, 35, 36, 37, 38, 39, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 20, 23, 26, 29, 22, 25, 28, 21, 24, 27, 30, 51, 52, 53, 54, 55, 56, 57, 58, 59, 40, 47, 44, 41, 48, 45, 42, 49, 46, 43, 50, 11, 12, 13, 14, 15, 16, 17, 18, 19]
        self._Y = [0, 41, 42, 43, 44, 45, 46, 47, 48, 49, 10, 17, 14, 11, 18, 15, 12, 19, 16, 13, 20, 1, 2, 3, 4, 5, 6, 7, 8, 9, 30, 33, 36, 39, 32, 35, 38, 31, 34, 37, 40, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 29, 28, 27, 26, 25, 24, 23, 22, 21]
        self._Z = [0, 3, 6, 9, 2, 5, 8, 1, 4, 7, 10, 43, 46, 49, 42, 45, 48, 41, 44, 47, 20, 13, 16, 19, 12, 15, 18, 11, 14, 17, 30, 23, 26, 29, 22, 25, 28, 21, 24, 27, 40, 33, 36, 39, 32, 35, 38, 31, 34, 37, 50, 57, 54, 51, 58, 55, 52, 59, 56, 53]

        self.reset()


    def reset(self):
        self.cube = deepcopy(self._map_st)

    def print(self):
        print(self.__str__())

    def __repr__(self):
        c = self.cube
        def C(n): return clr[c[n]].replace("x ", str(c[n]+100)[-2:])
        R= "\033[41m x \033[0m"
        G= "\033[42m x \033[0m"
        Y= "\033[43m x \033[0m"
        B= "\033[44m x \033[0m"
        O= "\033[40m x \033[0m"
        W= "\033[47m x \033[0m"

        clr = [W]*10 + [R]*10 + [B]*10 + [O]*10 + [G]*10 + [Y]*10
        sp = " "*9

        ret = ""
        ret+= (f"{sp}{C(31)}{C(32)}{C(33)}{sp}") + "\n"
        ret+= (f"{sp}{C(34)}{C(35)}{C(36)}{sp}") + "\n"
        ret+= (f"{sp}{C(37)}{C(38)}{C(39)}{sp}") + "\n"
        for i in [41,42,43,1,2,3,21,22,23]:
            ret+= (f"{C(i)}")
        ret+= "\n"
        for i in [44,45,46,4,5,6,24,25,26]:
            ret+= (f"{C(i)}")
        ret+= "\n"
        for i in [47,48,49,7,8,9,27,28,29]:
            ret+= (f"{C(i)}")
        ret+= "\n"
        ret+= (f"{sp}{C(11)}{C(12)}{C(13)}{sp}") + "\n"
        ret+= (f"{sp}{C(14)}{C(15)}{C(16)}{sp}") + "\n"
        ret+= (f"{sp}{C(17)}{C(18)}{C(19)}{sp}") + "\n"
        ret+= (f"{sp}{C(51)}{C(52)}{C(53)}{sp}") + "\n"
        ret+= (f"{sp}{C(54)}{C(55)}{C(56)}{sp}") + "\n"
        ret+= (f"{sp}{C(57)}{C(58)}{C(59)}{sp}") + "\n"
        return ret

    def l(self):
        self.cube = [self.cube[i] for i in self._l]
        self.moveslist += 'l '
    def r(self):
        self.cube = [self.cube[i] for i in self._r]
        self.moveslist += 'r '
    def u(self):
        self.cube = [self.cube[i] for i in self._u]
        self.moveslist += 'u '
    def d(self):
        self.cube = [self.cube[i] for i in self._d]
        self.moveslist += 'd '
    def f(self):
        self.cube = [self.cube[i] for i in self._f]
        self.moveslist += 'f '
    def b(self):
        self.cube = [self.cube[i] for i in self._b]
        self.moveslist += 'b '
    def x(self):
        self.cube = [self.cube[i] for i in self._x]
        self.moveslist += 'x '
    def y(self):
        self.cube = [self.cube[i] for i in self._y]
        self.moveslist += 'y '
    def z(self):
        self.cube = [self.cube[i] for i in self._z]
        self.moveslist += 'z '
    def L(self):
        self.cube = [self.cube[i] for i in self._L]
        self.moveslist += 'L '
    def R(self):
        self.cube = [self.cube[i] for i in self._R]
        self.moveslist += 'R '
    def U(self):
        self.cube = [self.cube[i] for i in self._U]
        self.moveslist += 'U '
    def D(self):
        self.cube = [self.cube[i] for i in self._D]
        self.moveslist += 'D '
    def F(self):
        self.cube = [self.cube[i] for i in self._F]
        self.moveslist += 'F '
    def B(self):
        self.cube = [self.cube[i] for i in self._B]
        self.moveslist += 'B '
    def X(self):
        self.cube = [self.cube[i] for i in self._X]
        self.moveslist += 'X '
    def Y(self):
        self.cube = [self.cube[i] for i in self._Y]
        self.moveslist += 'Y '
    def Z(self):
        self.cube = [self.cube[i] for i in self._Z]
        self.moveslist += 'Z '


    def move(self, m):
        ml = m.split()
        #print (ml)
        for i in ml:
            if i == "l": self.l()
            elif i == "r": self.r()
            elif i == "u": self.u()
            elif i == "d": self.d()
            elif i == "f": self.f()
            elif i == "b": self.b()
            elif i == "L": self.L()
            elif i == "R": self.R()
            elif i == "U": self.U()
            elif i == "D": self.D()
            elif i == "F": self.F()
            elif i == "B": self.B()
            elif i == "x": self.x()
            elif i == "y": self.y()
            elif i == "z": self.z()
            elif i == "X": self.X()
            elif i == "Y": self.Y()
            elif i == "Z": self.Z()

            else: print("error")
            #self.print()
        return (len(ml))


    tperm = "r u R U R f r r U R U r u R F"
    yperm = "r U R U r u R F r u R U R f r"

    def scramble(self, num =20):
        moves = ["R", "r", "L", "l", "F", "f", "B", "b", "U", "u", "D", "d"]
        orderlist = [moves[random.randint(0,11)] for i in range(num)]
        order = " ".join(orderlist)
        self.move(order)
        print (order)

    def reorrient(self):
        for i in (15,25,35,45,55):
            if self.cube[i] == 5:
                if i == 15:
                    self.x()
                elif i == 25:
                    self.y()
                elif i == 35:
                    self.X()
                elif i == 45:
                    self.Y()
                elif i == 55:
                    self.Y()
        for i in (25,35,45):
            if self.cube[i] == 15:
                if i == 25:
                    self.z()
                elif i == 35:
                    self.z()
                    self.z()
                elif i == 45:
                    self.Z()

    def movereverse(self, m):
        ml = m.split()
        mlreverse = []
        for i in range(len(ml)):
            mlreverse.append(ml[len(ml) - i-1])
        #print (ml)
        for i in mlreverse:
            if i ==   "l": self.L()
            elif i == "r": self.R()
            elif i == "u": self.U()
            elif i == "d": self.D()
            elif i == "f": self.F()
            elif i == "b": self.B()
            elif i == "L": self.l()
            elif i == "R": self.r()
            elif i == "U": self.u()
            elif i == "D": self.d()
            elif i == "F": self.f()
            elif i == "B": self.b()
            elif i == "X": self.x()
            elif i == "Y": self.y()
            elif i == "Z": self.z()
            elif i == "x": self.X()
            elif i == "y": self.Y()
            elif i == "z": self.Z()

            else: print("error", ml, mlreverse)
            #self.print()
        return (len(ml))

    edge_setup_moves = {
        2: "r X D l l",
        4: "L",
        6: "u Y u Y l",
        8: "r X d l l",
        12: "D l l",
        14: "l l",
        16: "d d l l",
        18: "d l l",
        #22: "buffer piece",
        24: "U y L",
        26: "u Y l",
        28: "d d L U y l",
        32: "r r x x D l l",
        34: "",
        #36: "buffer piece",
        38: "r r X X d l l",
        42: "l U y l",
        44: "u Y L",
        46: "U y l",
        48: "L U y l",
        52: "R x D l l",
        54: "l",
        56: "u u Y Y L",
        58: "R x d l l"
    }

    corner_setup_moves = {
        #31: "buffer piece",
        33: "r r",
        39: "f f d",
        37: "f f",
        #41: "buffer piece",
        43: "F d",
        49: "F",
        47: "D r",
        1: "f R",
        3: "R",
        9: "F R",
        7: "f f R",
        21: "f",
        23: "R f",
        29: "r r f",
        27: "r f",
        59: "r D",
        #57: "buffer piece",
        51: "d F",
        53: "r",
        11: "d",
        13: "",
        19: "D",
        17: "d d"
    }

    def nrml(self, movelist): # "non rational moves list"
        facevalues = {
            5 : 'w',
            15 : 'r',
            25 : 'b',
            35 : 'o',
            45 : 'g',
            55 : 'y'}
        ret = ""
        movelist = movelist.split()
        ff = facevalues[self.cube[5]]   # current face
        df = facevalues[self.cube[15]] # current bottom face

        m = {'w': {'r': {'r': 'r','l': 'l','f': 'f','b': 'b','u': 'u','d': 'd','R': 'R','L': 'L','F': 'F','B': 'B','U': 'U','D': 'D'},'o': {'r': 'l','l': 'r','f': 'f','b': 'b','u': 'd','d': 'u','R': 'L','L': 'R','F': 'F','B': 'B','U': 'D','D': 'U'},'b': {'r': 'u','l': 'd','f': 'f','b': 'b','u': 'l','d': 'r','R': 'U','L': 'D','F': 'F','B': 'B','U': 'L','D': 'R'},'g': {'r': 'd','l': 'u','f': 'f','b': 'b','u': 'r','d': 'l','R': 'D','L': 'U','F': 'F','B': 'B','U': 'R','D': 'L'}},
            'y': {'r': {'r': 'l','l': 'r','f': 'b','b': 'f','u': 'u','d': 'd','R': 'L','L': 'R','F': 'B','B': 'F','U': 'U','D': 'D'},'o': {'r': 'r','l': 'l','f': 'b','b': 'f','u': 'd','d': 'u','R': 'R','L': 'L','F': 'B','B': 'F','U': 'D','D': 'U'},'b': {'r': 'd','l': 'u','f': 'b','b': 'f','u': 'l','d': 'r','R': 'D','L': 'U','F': 'B','B': 'F','U': 'L','D': 'R'},'g': {'r': 'u','l': 'd','f': 'b','b': 'f','u': 'r','d': 'l','R': 'U','L': 'D','F': 'B','B': 'F','U': 'R','D': 'L'}},
            'r': {'w': {'r': 'l','l': 'r','f': 'd','b': 'u','u': 'b','d': 'f','R': 'L','L': 'R','F': 'D','B': 'U','U': 'B','D': 'F'},'y': {'r': 'r','l': 'l','f': 'd','b': 'u','u': 'f','d': 'b','R': 'R','L': 'L','F': 'D','B': 'U','U': 'F','D': 'B'},'b': {'r': 'f','l': 'b','f': 'd','b': 'u','u': 'l','d': 'r','R': 'F','L': 'B','F': 'D','B': 'U','U': 'L','D': 'R'},'g': {'r': 'b','l': 'f','f': 'd','b': 'u','u': 'r','d': 'l','R': 'B','L': 'F','F': 'D','B': 'U','U': 'R','D': 'L'}},
            'o': {'w': {'r': 'r','l': 'l','f': 'u','b': 'd','u': 'b','d': 'f','R': 'R','L': 'L','F': 'U','B': 'D','U': 'B','D': 'F'},'y': {'r': 'l','l': 'r','f': 'u','b': 'd','u': 'f','d': 'b','R': 'L','L': 'R','F': 'U','B': 'D','U': 'F','D': 'B'},'b': {'r': 'b','l': 'f','f': 'u','b': 'd','u': 'l','d': 'r','R': 'B','L': 'F','F': 'U','B': 'D','U': 'L','D': 'R'},'g': {'r': 'f','l': 'b','f': 'u','b': 'd','u': 'r','d': 'l','R': 'F','L': 'B','F': 'U','B': 'D','U': 'R','D': 'L'}},
            'b': {'w': {'r': 'd','l': 'u','f': 'r','b': 'l','u': 'b','d': 'f','R': 'D','L': 'U','F': 'R','B': 'L','U': 'B','D': 'F'},'y': {'r': 'u','l': 'd','f': 'r','b': 'l','u': 'f','d': 'b','R': 'U','L': 'D','F': 'R','B': 'L','U': 'F','D': 'B'},'r': {'r': 'b','l': 'f','f': 'r','b': 'l','u': 'u','d': 'd','R': 'B','L': 'F','F': 'R','B': 'L','U': 'U','D': 'D'},'o': {'r': 'f','l': 'b','f': 'r','b': 'l','u': 'd','d': 'u','R': 'F','L': 'B','F': 'R','B': 'L','U': 'D','D': 'U'}},
            'g': {'w': {'r': 'u','l': 'd','f': 'l','b': 'r','u': 'b','d': 'f','R': 'U','L': 'D','F': 'L','B': 'R','U': 'B','D': 'F'},'y': {'r': 'd','l': 'u','f': 'l','b': 'r','u': 'f','d': 'b','R': 'D','L': 'U','F': 'L','B': 'R','U': 'F','D': 'B'},'r': {'r': 'f','l': 'b','f': 'l','b': 'r','u': 'u','d': 'd','R': 'F','L': 'B','F': 'L','B': 'R','U': 'U','D': 'D'},'o': {'r': 'b','l': 'f','f': 'l','b': 'r','u': 'd','d': 'u','R': 'B','L': 'F','F': 'L','B': 'R','U': 'D','D': 'U'}}}

        n1 = {'w': {'r': {'x': 'r', 'y': 'b', 'z': 'w', 'X': 'o', 'Y': 'g', 'Z': 'w'},'o': {'x': 'o', 'y': 'g', 'z': 'w', 'X': 'r', 'Y': 'b', 'Z': 'w'},'b': {'x': 'b', 'y': 'o', 'z': 'w', 'X': 'g', 'Y': 'r', 'Z': 'w'},'g': {'x': 'g', 'y': 'r', 'z': 'w', 'X': 'b', 'Y': 'o', 'Z': 'w'}},
            'y': {'r': {'x': 'r', 'y': 'g', 'z': 'y', 'X': 'o', 'Y': 'b', 'Z': 'y'},'o': {'x': 'o', 'y': 'b', 'z': 'y', 'X': 'r', 'Y': 'g', 'Z': 'y'},'b': {'x': 'b', 'y': 'r', 'z': 'y', 'X': 'g', 'Y': 'o', 'Z': 'y'},'g': {'x': 'g', 'y': 'o', 'z': 'y', 'X': 'b', 'Y': 'r', 'Z': 'y'}},
            'r': {'w': {'x': 'w', 'y': 'g', 'z': 'r', 'X': 'y', 'Y': 'b', 'Z': 'r'},'y': {'x': 'y', 'y': 'b', 'z': 'r', 'X': 'w', 'Y': 'g', 'Z': 'r'},'b': {'x': 'b', 'y': 'w', 'z': 'r', 'X': 'g', 'Y': 'y', 'Z': 'r'},'g': {'x': 'g', 'y': 'y', 'z': 'r', 'X': 'b', 'Y': 'w', 'Z': 'r'}},
            'o': {'w': {'x': 'w', 'y': 'b', 'z': 'o', 'X': 'y', 'Y': 'g', 'Z': 'o'},'y': {'x': 'y', 'y': 'g', 'z': 'o', 'X': 'w', 'Y': 'b', 'Z': 'o'},'b': {'x': 'b', 'y': 'y', 'z': 'o', 'X': 'g', 'Y': 'w', 'Z': 'o'},'g': {'x': 'g', 'y': 'w', 'z': 'o', 'X': 'b', 'Y': 'y', 'Z': 'o'}},
            'b': {'w': {'x': 'w', 'y': 'r', 'z': 'b', 'X': 'y', 'Y': 'o', 'Z': 'b'},'y': {'x': 'y', 'y': 'o', 'z': 'b', 'X': 'w', 'Y': 'r', 'Z': 'b'},'r': {'x': 'r', 'y': 'y', 'z': 'b', 'X': 'o', 'Y': 'w', 'Z': 'b'},'o': {'x': 'o', 'y': 'w', 'z': 'b', 'X': 'r', 'Y': 'y', 'Z': 'b'}},
            'g': {'w': {'x': 'w', 'y': 'o', 'z': 'g', 'X': 'y', 'Y': 'r', 'Z': 'g'},'y': {'x': 'y', 'y': 'r', 'z': 'g', 'X': 'w', 'Y': 'o', 'Z': 'g'},'r': {'x': 'r', 'y': 'w', 'z': 'g', 'X': 'o', 'Y': 'y', 'Z': 'g'},'o': {'x': 'o', 'y': 'y', 'z': 'g', 'X': 'r', 'Y': 'w', 'Z': 'g'}}}

        n2 = {'w': {'r': {'x': 'y', 'y': 'r', 'z': 'b', 'X': 'w', 'Y': 'r', 'Z': 'g'},'o': {'x': 'y', 'y': 'o', 'z': 'g', 'X': 'w', 'Y': 'o', 'Z': 'b'},'b': {'x': 'y', 'y': 'b', 'z': 'o', 'X': 'w', 'Y': 'b', 'Z': 'r'},'g': {'x': 'y', 'y': 'g', 'z': 'r', 'X': 'w', 'Y': 'g', 'Z': 'o'}},
            'y': {'r': {'x': 'w', 'y': 'r', 'z': 'g', 'X': 'y', 'Y': 'r', 'Z': 'b'},'o': {'x': 'w', 'y': 'o', 'z': 'b', 'X': 'y', 'Y': 'o', 'Z': 'g'},'b': {'x': 'w', 'y': 'b', 'z': 'r', 'X': 'y', 'Y': 'b', 'Z': 'o'},'g': {'x': 'w', 'y': 'g', 'z': 'o', 'X': 'y', 'Y': 'g', 'Z': 'r'}},
            'r': {'w': {'x': 'o', 'y': 'w', 'z': 'g', 'X': 'r', 'Y': 'w', 'Z': 'b'},'y': {'x': 'o', 'y': 'y', 'z': 'b', 'X': 'r', 'Y': 'y', 'Z': 'g'},'b': {'x': 'o', 'y': 'b', 'z': 'w', 'X': 'r', 'Y': 'b', 'Z': 'y'},'g': {'x': 'o', 'y': 'g', 'z': 'y', 'X': 'r', 'Y': 'g', 'Z': 'w'}},
            'o': {'w': {'x': 'r', 'y': 'w', 'z': 'b', 'X': 'o', 'Y': 'w', 'Z': 'g'},'y': {'x': 'r', 'y': 'y', 'z': 'g', 'X': 'o', 'Y': 'y', 'Z': 'b'},'b': {'x': 'r', 'y': 'b', 'z': 'y', 'X': 'o', 'Y': 'b', 'Z': 'w'},'g': {'x': 'r', 'y': 'g', 'z': 'w', 'X': 'o', 'Y': 'g', 'Z': 'y'}},
            'b': {'w': {'x': 'g', 'y': 'w', 'z': 'r', 'X': 'b', 'Y': 'w', 'Z': 'o'},'y': {'x': 'g', 'y': 'y', 'z': 'o', 'X': 'b', 'Y': 'y', 'Z': 'r'},'r': {'x': 'g', 'y': 'r', 'z': 'y', 'X': 'b', 'Y': 'r', 'Z': 'w'},'o': {'x': 'g', 'y': 'o', 'z': 'w', 'X': 'b', 'Y': 'o', 'Z': 'y'}},
            'g': {'w': {'x': 'b', 'y': 'w', 'z': 'o', 'X': 'g', 'Y': 'w', 'Z': 'r'},'y': {'x': 'b', 'y': 'y', 'z': 'r', 'X': 'g', 'Y': 'y', 'Z': 'o'},'r': {'x': 'b', 'y': 'r', 'z': 'w', 'X': 'g', 'Y': 'r', 'Z': 'y'},'o': {'x': 'b', 'y': 'o', 'z': 'y', 'X': 'g', 'Y': 'o', 'Z': 'w'}}}

        for i in movelist:
            cff = deepcopy(ff)
            cdf = deepcopy(df)
            ci = deepcopy(i)
            if i in ['x','y','z','X','Y','Z']:
                ff = n1[cff][cdf][ci]
                df = n2[cff][cdf][ci]
            elif i in ["R", "r", "L", "l", "F", "f", "B", "b", "U", "u", "D", "d"]:
                ret += m[cff][cdf][ci]
        print(ret)
        return(ret)



    def solve(self):
        self.moveslist = ""
        #c = self.cube
        self.reorrient()
        nom = self.solve_edges() + self.solve_corners()
        print('ml', (deepcopy(self.moveslist)))
        return(nom, self.nrml(deepcopy(self.moveslist)))

    def check_complete_edges_solve(self):
        for i in range(0, 60, 2):
            if self.cube[i] != i:
                return False
        return True

    def check_complete_corners_solve(self):
        for i in range(1, 60, 2):
            if self.cube[i] != i:
                return False
        return True

    def solve_edges(self):
        nom = 0
        count = 0
        while count < 500:
            count += 1
            print(f"{count} ")
            print(nom)
            if (self.cube[36] == 36 or self.cube[36] == 22):
                count += 1
                if self.check_complete_edges_solve():
                    return nom

                # swap 36/22 with a wrong edge
                for i in range (0,60,2):
                    if i%10 !=0 and self.cube[i] not in [22, 36] and i != self.cube[i]:
                        wrong = i
                        break
                setupmoves = self.edge_setup_moves[wrong]
                nom += self.move(setupmoves)
                nom += self.move(self.tperm)
                nom += self.movereverse(setupmoves)
                self.print()

            setupmoves = self.edge_setup_moves[self.cube[36]]
            nom += self.move(setupmoves)
            nom += self.move(self.tperm)
            nom += self.movereverse(setupmoves)
            self.print()

    def solve_corners(self):
        nom = 0
        count = 0
        while count < 500:
            count+= 1
            print(f"{count} ")

            if self.cube[41] == 41 or self.cube[41] == 31 or self.cube[41] == 57:
                count += 1
                if self.check_complete_corners_solve():
                    return nom

                #swap 41/31/57 with wrong edge
                for i in range (1,62,2):
                    if i%5 != 0 and self.cube[i] not in [41,31,57] and i != self.cube[i]:
                        wrong = i
                        break
                setupmoves = self.corner_setup_moves[wrong]
                nom += self.move(setupmoves)
                nom += self.move(self.yperm)
                nom += self.movereverse(setupmoves)
                self.print()

            setupmoves = self.corner_setup_moves[self.cube[41]]
            nom += self.move(setupmoves)
            nom += self.move(self.yperm)
            nom += self.movereverse(setupmoves)
            self.print()

    def configure(self, configuration):
        state = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
        #configuration check and sort
        config = []
        convert = {
            0:1,1:2,2:3,3:4,4:5,5:6,6:7,7:8,8:9,9:11,10:12,11:13,12:14,13:15,14:16,15:17,16:18,17:19,18:21,19:22,20:23,21:24,22:25,23:26,24:27,25:28,26:29,27:31,28:32,29:33,30:34,31:35,32:36,33:37,34:38,35:39,36:41,37:42,38:43,39:44,40:45,41:46,42:47,43:48,44:49,45:51,46:52,47:53,48:54,49:55,50:56,51:57,52:58,53:59}
        for i in configuration:
            config.append(i)
        if sorted(config) != ['b','b','b','b','b','b','b','b','b','g','g','g','g','g','g','g','g','g','o','o','o','o','o','o','o','o','o','r','r','r','r','r','r','r','r','r','w','w','w','w','w','w','w','w','w','y','y','y','y','y','y','y','y','y']:
            print ("error, configuration example : wwwwwwwwwrrrrrrrrrbbbbbbbbbooooooooogggggggggyyyyyyyyy")

        #centers
        x = [[4,5],[13,15],[22,25],[31,35],[40,45],[49,55]]
        centredict = {
            'w': 5,
            'r': 15,
            'b': 25,
            'o': 35,
            'g': 45,
            'y': 55}
        for i in x:
            state [(i[1])] = centredict[config[(i[0])]]
        #corners
        x = [(0, 33, 38), (2, 35, 18), (6, 44, 9), (8, 24, 11), (51, 27, 36), (53, 29, 20), (45, 15, 42), (47, 17, 26)]
        for i,j,k in x:
            if sorted([config[i],config[j],config[k]]) == sorted(['w','o','g']):
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('w'))])]] = 1
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('o'))])]] = 37
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('g'))])]] = 43
            elif sorted([config[i],config[j],config[k]]) == sorted(['w','o','b']):
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('w'))])]] = 3
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('o'))])]] = 39
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('b'))])]] = 21
            elif sorted([config[i],config[j],config[k]]) == sorted(['w','r','g']):
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('w'))])]] = 7
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('r'))])]] = 11
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('g'))])]] = 49
            elif sorted([config[i],config[j],config[k]]) == sorted(['w','r','b']):
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('w'))])]] = 9
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('r'))])]] = 13
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('b'))])]] = 27
            elif sorted([config[i],config[j],config[k]]) == sorted(['y','o','g']):
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('y'))])]] = 57
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('o'))])]] = 31
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('g'))])]] = 41
            elif sorted([config[i],config[j],config[k]]) == sorted(['y','o','b']):
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('y'))])]] = 59
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('o'))])]] = 33
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('b'))])]] = 23
            elif sorted([config[i],config[j],config[k]]) == sorted(['y','r','g']):
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('y'))])]] = 51
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('r'))])]] = 17
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('g'))])]] = 47
            elif sorted([config[i],config[j],config[k]]) == sorted(['y','r','b']):
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('y'))])]] = 53
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('r'))])]] = 19
                state [convert[([i,j,k][([config[i],config[j],config[k]].index('b'))])]] = 29
        #edges
        x = [(1,34),(5,21),(3,41),(7,10),(12,43),(14,25),(16,46),(19,32),(30,37),(28,52),(50,23),(48,39)]
        for i,j in x:
            if sorted([config[i],config[j]]) == sorted(['w','o']):
                state [convert[([i,j][([config[i],config[j]].index('w'))])]] = 2
                state [convert[([i,j][([config[i],config[j]].index('o'))])]] = 38
            elif sorted([config[i],config[j]]) == sorted(['w','g']):
                state [convert[([i,j][([config[i],config[j]].index('w'))])]] = 4
                state [convert[([i,j][([config[i],config[j]].index('g'))])]] = 46
            elif sorted([config[i],config[j]]) == sorted(['w','r']):
                state [convert[([i,j][([config[i],config[j]].index('w'))])]] = 8
                state [convert[([i,j][([config[i],config[j]].index('r'))])]] = 12
            elif sorted([config[i],config[j]]) == sorted(['w','b']):
                state [convert[([i,j][([config[i],config[j]].index('w'))])]] = 6
                state [convert[([i,j][([config[i],config[j]].index('b'))])]] = 24
            elif sorted([config[i],config[j]]) == sorted(['y','b']):
                state [convert[([i,j][([config[i],config[j]].index('y'))])]] = 56
                state [convert[([i,j][([config[i],config[j]].index('b'))])]] = 26
            elif sorted([config[i],config[j]]) == sorted(['y','g']):
                state [convert[([i,j][([config[i],config[j]].index('y'))])]] = 54
                state [convert[([i,j][([config[i],config[j]].index('g'))])]] = 44
            elif sorted([config[i],config[j]]) == sorted(['y','o']):
                state [convert[([i,j][([config[i],config[j]].index('y'))])]] = 58
                state [convert[([i,j][([config[i],config[j]].index('o'))])]] = 32
            elif sorted([config[i],config[j]]) == sorted(['y','r']):
                state [convert[([i,j][([config[i],config[j]].index('y'))])]] = 52
                state [convert[([i,j][([config[i],config[j]].index('r'))])]] = 18
            elif sorted([config[i],config[j]]) == sorted(['o','b']):
                state [convert[([i,j][([config[i],config[j]].index('o'))])]] = 36
                state [convert[([i,j][([config[i],config[j]].index('b'))])]] = 22
            elif sorted([config[i],config[j]]) == sorted(['o','g']):
                state [convert[([i,j][([config[i],config[j]].index('o'))])]] = 34
                state [convert[([i,j][([config[i],config[j]].index('g'))])]] = 42
            elif sorted([config[i],config[j]]) == sorted(['r','b']):
                state [convert[([i,j][([config[i],config[j]].index('r'))])]] = 16
                state [convert[([i,j][([config[i],config[j]].index('b'))])]] = 28
            elif sorted([config[i],config[j]]) == sorted(['r','g']):
                state [convert[([i,j][([config[i],config[j]].index('r'))])]] = 14
                state [convert[([i,j][([config[i],config[j]].index('g'))])]] = 48
        self.cube = state

In [34]:
cb = rubik

In [40]:
y = cb()

## Serial

In [ ]:
def sendserial(msg, serial_port = 'COM3', baud_rate = 9600, delay=0):
    ser = serial.Serial(serial_port, baud_rate, timeout=1)
    ser.write(msg.encode())
    time.sleep(delay)
    print("",end=".")
    ser.close()

In [63]:
sendserial('llruRURfrrURUruRFLLrFllrbRBRurrBRBrbRULLfRRBllrfRFRdrrFRFrfRDLLbrrFllrbRBRurrBRBrbRULLfRLruRURfrrURUruRFluuRluLULbllULUluLBrUULruRURfrrURUruRFluurluLULbllULUluLBRUUlruRURfrrURUruRFLuurluLULbllULUluLBRUUrfllrbRBRurrBRBrbRULLFRRbllrfRFRdrrFRFrfRDLLBrrfllrbRBRurrBRBrbRULLFRddllruRURfrrURUruRFLLDDruRURfrrURUruRFddllruRURfrrURUruRFLLDD')

.

## Capturing & Calibration

In [ ]:
cap_rects = [(285, 160, 315, 180), #corner
             (310, 200, 330, 230), #corner
             (270, 200, 290, 230), #corner
             (310, 260, 330, 290), #edge
             (270, 260, 290, 290)] #edge

In [ ]:
def draw_guide(frame):
    global cap_rects
    line_clr = (255,0,255) #pink
    rect_clr = (255,0,255) #pink
    rect_thick = 2
    
    cv2.line(frame,(300, 200),(300,311),line_clr,2)
    cv2.line(frame,(300, 200),(100,90),line_clr,2)
    cv2.line(frame,(300, 200),(500,85),line_clr,2)
    
    for r in cap_rects:
        cv2.rectangle(frame, r[:2], r[2:], line_clr,rect_thick)


In [ ]:
def cap_frame(show_guide=True):
    cap = cv2.VideoCapture(0)

    while(True):
        ret, frame = cap.read()
        if(ret):
            if show_guide:
                guide_frame = deepcopy(frame)
                draw_guide(guide_frame)
            cv2.imshow("output", guide_frame)
            if cv2.waitKey(1) & 0xFF == ord('c'):
                break
        else:
            break
        
    cv2.destroyAllWindows()
    cap.release()
    return frame

In [ ]:
def capture(cells):
    global cap_rects
    ret = []
    cap = cv2.VideoCapture(0)
    rd = False
    while(not rd):
        rd, img = cap.read()
    
    
    for i in cells:
        cell = img[cap_rects[i][1]:cap_rects[i][3], cap_rects[i][0]: cap_rects[i][2]]
        # cell = rgb2hsv(cell)
        mean = (np.mean(cell, axis=(0, 1)).round().astype(int)).tolist()
        ret.append(mean)
    return(ret)

In [ ]:
def display_calib(f = "calib.json"):
    calib = json.load(open(f))
    for ind, j in enumerate(calib):
        print("position", ind)
        for i in ['w', 'y', 'r', 'o', 'b', 'g']:
            print(i, j[i])
            x = [j[i][2],j[i][1],j[i][0]]
            plt.imshow(np.array([[x]]))
            plt.show()

### Calibration

In [ ]:
def calibrate ():
    _ = cap_frame()
    delay = 1
    p0 = {'w':None, 'r':None, 'b':None, 'o':None, 'g':None, 'y':None}
    p1 = {'w':None, 'r':None, 'b':None, 'o':None, 'g':None, 'y':None}
    p2 = {'w':None, 'r':None, 'b':None, 'o':None, 'g':None, 'y':None}
    p3 = {'w':None, 'r':None, 'b':None, 'o':None, 'g':None, 'y':None}
    p4 = {'w':None, 'r':None, 'b':None, 'o':None, 'g':None, 'y':None}
    calib = [p0, p1, p2, p3, p4]


    #CORNER CALIB

    l,m,n = "o", "w", "g"
    for i in [0,1,2]:
        p0[l], p1[m], p2[n] = capture((0,1,2))
        sendserial("l u L U " * 2, delay=delay)
        l,m,n =  m,n,l

    sendserial("r r u", delay=delay)

    l,m,n = "r", "b", "y"
    for i in [0,1,2]:
        p0[l], p1[m], p2[n] = capture((0,1,2))
        sendserial("l u L U " * 2, delay=delay)
        l,m,n =  m,n,l

    sendserial("U r r", delay=delay)

    #EDGE CALIB


    p3['w'], p4['g'] = capture((3,4))
    sendserial("L U F", delay=delay)
    p3['g'], p4['w'] = capture((3,4))
    sendserial("f u l d L", delay=delay)


    p3['r'], p4['y'] = capture((3,4))
    sendserial("L U F", delay=delay)
    p3['y'], p4['r'] = capture((3,4))
    sendserial("f u l l D u F", delay=delay)

    p3['b'], p4['o'] = capture((3,4))
    sendserial("L U F", delay=delay)
    p3['o'], p4['b'] = capture((3,4))
    sendserial("f u l f U", delay=delay)


    json.dump(calib, open("calib.json", "w"), indent = 4)


    # cropped_img = frame[y[c]:y[c]+h[c], x[c]:x[c]+w[c]]
    # # colours[order[c]] = cropped_img
    # p0m = (np.mean(rgb2hsv(cropped_img), axis=(0, 1)).round().astype(int)).tolist()



In [ ]:
calibrate()
display_calib()

## COMPLETE CALIBRATION DONE 

## Image capture

## colour detection

def cube_scan():


In [91]:
def cube_scan():
    _ = cap_frame()
    delay = 1

    calib = json.load(open("calib.json"))


    configuration = list(54*'*')

    configuration[4] = 'w'
    configuration[13] = 'r'
    configuration[22] = 'b'
    configuration[31] = 'o'
    configuration[40] = 'g'
    configuration[49] = 'y'


    def solve(pos, colour):
        global calib
        pos_req = calib[pos]
        closest_d = 1000000000
        closest_c = None
        # print(colour)
        # print(pos_req)
        # print(pos_req.items())
        for clr, val in pos_req.items():
            # print(colour[0])
            # print(val[0])
            dist = (colour[0][0]-val[0])*(colour[0][0]-val[0]) + (colour[0][1]-val[1])*(colour[0][1]-val[1]) + (colour[0][2]-val[2])*(colour[0][2]-val[2])
            # print(dist,clr)
            if dist < closest_d:
                # print("asgn", dist,clr)

                closest_d = dist
                closest_c = clr

        # print("st",closest_c)
        # print("dist",closest_d)

        return(closest_c)



    positions = {
        37:0,
        49:0,
        13:0,
        21:0,
        17:0,
        41:0,
        33:0,
        29:0,
        0 :1,
        5 :1,
        7 :1,
        2 :1,
        51:1,
        57:1,
        59:1,
        53:1,
        43:2,
        11:2,
        27:2,
        39:2,
        47:2,
        31:2,
        23:2,
        19:2,
        3 :3,
        6 :3,
        4 :3,
        1 :3,
        54:3,
        52:3,
        56:3,
        58:3,
        34:3,
        14:3,
        28:3,
        22:3,
        46:4,
        12:4,
        24:4,
        38:4,
        44:4,
        18:4,
        26:4,
        32:4,
        42:4,
        48:4,
        16:4,
        36:4}

    # FRONT LAYER:


    configuration[33], configuration[0], configuration[38], configuration[3], configuration[41] = solve(0, capture([0])), solve(1, capture([1])), solve(2, capture([2])), solve(3, capture([3])), solve(4, capture([4])) 
    sendserial("f", delay=delay)
    configuration[44], configuration[6], configuration[9], configuration[7], configuration[10] = solve(0, capture([0])), solve(1, capture([1])), solve(2, capture([2])), solve(3, capture([3])), solve(4, capture([4])) 
    sendserial("f")
    configuration[11], configuration[8], configuration[24], configuration[5], configuration[21] = solve(0, capture([0])), solve(1, capture([1])), solve(2, capture([2])), solve(3, capture([3])), solve(4, capture([4])) 
    sendserial("f")
    configuration[18], configuration[2], configuration[35], configuration[1], configuration[34] = solve(0, capture([0])), solve(1, capture([1])), solve(2, capture([2])), solve(3, capture([3])), solve(4, capture([4])) 
    sendserial("f")

    # BACK LAYER:

    sendserial("ddrr", delay=delay)
    sendserial("lluu", delay=delay)
    configuration[15], configuration[45], configuration[42], configuration[48], configuration[39] = solve(0, capture([0])), solve(1, capture([1])), solve(2, capture([2])), solve(3, capture([3])), solve(4, capture([4])) 
    sendserial("f", delay=delay)
    configuration[36], configuration[51], configuration[27], configuration[46], configuration[16] = solve(0, capture([0])), solve(1, capture([1])), solve(2, capture([2])), solve(3, capture([3])), solve(4, capture([4])) 
    sendserial("f", delay=delay)
    configuration[29], configuration[53], configuration[20], configuration[50], configuration[23] = solve(0, capture([0])), solve(1, capture([1])), solve(2, capture([2])), solve(3, capture([3])), solve(4, capture([4])) 
    sendserial("f", delay=delay)
    configuration[26], configuration[47], configuration[17], configuration[52], configuration[28] = solve(0, capture([0])), solve(1, capture([1])), solve(2, capture([2])), solve(3, capture([3])), solve(4, capture([4])) 
    sendserial("f", delay=delay)
    sendserial("uull", delay=delay)
    sendserial("rrdd", delay=delay)

    # EDGES:

    sendserial("l", delay=delay)
    configuration[30], configuration[37] = solve(3, capture([3])), solve(4, capture([4])) 
    sendserial("ll", delay=delay)
    configuration[12], configuration[43] = solve(3, capture([3])), solve(4, capture([4])) 
    sendserial("luDf", delay=delay)
    configuration[25], configuration[14] = solve(3, capture([3])), solve(4, capture([4])) 
    sendserial("ff", delay=delay)
    configuration[19], configuration[32] = solve(3, capture([3])), solve(4, capture([4])) 
    sendserial("fUd", delay=delay)

    final = ""
    for i in configuration:
        final += i
    final
    return(final)





In [94]:
final = cube_scan()

.................

In [95]:
final

'ybwwwwwwwrrrrrorrrbyybbbbbbooooooorowwbggggggyyyyyyggg'

In [115]:
c = rubik()

In [116]:
c.configure(final)

In [117]:
c

          37 34 31         
          38 35 32         
          33 16 39         
 01 02 23 59 28 03 21 58 57
 44 45 46 04 05 06 24 25 26
 47 48 49 07 08 09 27 22 29
          11 12 13         
          14 15 36         
          17 18 19         
          51 52 53         
          54 55 56         
          43 42 41         

In [106]:
x = c.solve()

1 
0
          37 32 39         
          38 35 34         
          33 16 31         
 01 02 23 59 28 57 41 42 03
 44 45 46 04 05 06 24 25 26
 47 48 49 07 08 09 27 22 29
          11 12 13         
          14 15 36         
          17 18 19         
          51 52 53         
          54 55 56         
          43 58 21         

2 
29
          37 32 31         
          34 35 38         
          33 16 39         
 01 42 23 59 28 03 21 02 57
 44 45 46 04 05 06 24 25 26
 47 48 49 07 08 09 27 22 29
          11 12 13         
          14 15 36         
          17 18 19         
          51 52 53         
          54 55 56         
          43 58 41         

3 
44
          37 32 39         
          34 35 16         
          33 38 31         
 01 42 23 59 02 57 41 28 03
 44 45 46 04 05 06 24 25 26
 47 48 49 07 08 09 27 22 29
          11 12 13         
          14 15 36         
          17 18 19         
          51 52 53         
          54 55 56         
 

In [107]:
x[1], len(x[1])

('rrUllrdRDRbrrDRDrdRBLLuRRruRURfrrURUruRFrrullrdRDRbrrDRDrdRBLLURRddllruRURfrrURUruRFLLDDfRrURUruRFruRURfrrFrDrURUruRFruRURfrdR',
 126)

In [108]:
y = " ".join(list(x[1]))
y

'r r U l l r d R D R b r r D R D r d R B L L u R R r u R U R f r r U R U r u R F r r u l l r d R D R b r r D R D r d R B L L U R R d d l l r u R U R f r r U R U r u R F L L D D f R r U R U r u R F r u R U R f r r F r D r U R U r u R F r u R U R f r d R'

In [127]:
sendserial('F r D r U R U r u R F r u R U R f r d R')

sendserial('rDRFrurURfrURuruRdRf')

.

In [118]:
c.move(y[:-2])

125

In [126]:
c.movereverse('F r D r U R U r u R F r u R U R f r d R')

20

In [124]:
x[1][97:]

'FruRURfrrFrDrURUruRFruRURfrdR'